# Aging Report

Below you see the code to retrieve the aging report from the Docker container test_sshd2.

See also the example code on [AppliAR.jl](https://www.appligate.nl/AppliAR.jl/stable/chapter4/).

### Activate the local environment in the TestAppliAR folder

The application uses the packages:
- AppliSales
- AppliGeneralLegder
- AppliAR (add https://github.com/rbontekoe/AppliAR.jl)
- DataFrames
- Rocket
- Query

In [1]:
using Pkg; Pkg.activate(".");

 Activating environment at `~/projects/TestAppliAR/Project.toml`


### Load [Rocket.jl](https://github.com/biaslab/Rocket.jl), the actor package

In [2]:
using Rocket

### Restart the containers test_sshd and test_sshd2

See section 3 `creating the containers` of the course: [BAWJ](https://www.appligate.nl/BAWJ/stable/chapter13/)

In [3]:
cmd = `docker stop test_sshd` # General Ledger
run(cmd);
cmd = `docker stop test_sshd2` # Accounts Receivable
run(cmd);
cmd = `docker start test_sshd`
run(cmd);
cmd = `docker start test_sshd2`
run(cmd);

test_sshd
test_sshd2
test_sshd
test_sshd2


### Allow distributed computation

In [4]:
using Distributed

### Connect to the containes test_sshd (General Ledger) and test_sshd2 (Accounts Receivable)

In [5]:
addprocs([("rob@172.17.0.2", 1)]; exeflags=`--project=$(Base.active_project())`, tunnel=true, dir="/home/rob")
addprocs([("rob@172.17.0.3", 1)]; exeflags=`--project=$(Base.active_project())`, tunnel=true, dir="/home/rob")

1-element Array{Int64,1}:
 3

### Assign the two processes

In [6]:
gl_pid = procs()[2] # general ledger
ar_pid = procs()[3] # accounts receivable (invoices/bankstatements)

3

### Load the packages

In [7]:
@everywhere using AppliSales
@everywhere using AppliGeneralLedger
@everywhere using AppliAR
@everywhere using Query

### Load the actors

See: [actors.jl](https://www.appligate.nl/AppliAR.jl/stable/chapter4/#actors.jl)

In [8]:
include("./src/actors.jl");

In [9]:
sales_actor = SalesActor()
ar_actor = ARActor(ar_pid)
gl_actor = GLActor(gl_pid)
stm_actor = StmActor();

### Start the application

                           StmActor
                              |
                              | BankStatement(s)
                              ↓       
       SalesActor -------> ARActor -------> GLActor
                  Order(s)    ↑    Entry(s)    ↑
                              ↓                ↓
                            Store            Store

The SalesActor sends a message with three orders to the ARActor (Accounts Receivable Actor). The ARActor creates three invoices and sends the journal entries to the GLActor (General Ledger Actor). 

In [10]:
AppliSales.process() # The three orders

3-element Array{AppliSales.Order,1}:
 AppliSales.Order("670402629847452872", AppliSales.Organization("7769092163788803777", "Scrooge Investment Bank", "1180 Seven Seas Dr", "FL 32830", "Lake Buena Vista", "USA"), AppliSales.Training("LS", Dates.DateTime("2019-08-30T00:00:00"), 2, "Learn Smiling", 1000.0), "PO-456", "Scrooge McDuck", "scrooge@duckcity.com", ["Scrooge McDuck"])
 AppliSales.Order("10551933522330692664", AppliSales.Organization("5619503041329077350", "Duck City Chronicals", "1185 Seven Seas Dr", "FL 32830", "Lake Buena Vista", "USA"), AppliSales.Training("LS", Dates.DateTime("2019-08-30T00:00:00"), 2, "Learn Smiling", 1000.0), "DD-001", "Mickey Mouse", "mickey@duckcity.com", ["Mini Mouse", "Goofy"])
 AppliSales.Order("636974396306383826", AppliSales.Organization("10561277296584314514", "Donalds Hardware Store", "1190 Seven Seas Dr", "FL 32830", "Lake Buena Vista", "USA"), AppliSales.Training("LS", Dates.DateTime("2019-08-30T00:00:00"), 2, "Learn Smiling", 1000.0), "", "Don

In [11]:
subscribe!(from(["START"]), sales_actor); # Start the application

┌ Info: GLActor completed!
└ @ Main /home/rob/projects/TestAppliAR/src/actors.jl:54
┌ Info: GLActor completed!
└ @ Main /home/rob/projects/TestAppliAR/src/actors.jl:54
┌ Info: GLActor completed!
└ @ Main /home/rob/projects/TestAppliAR/src/actors.jl:54
┌ Info: ARActor Completed!
└ @ Main /home/rob/projects/TestAppliAR/src/actors.jl:41
┌ Info: SalesActor completed!
└ @ Main /home/rob/projects/TestAppliAR/src/actors.jl:24


### Process the two bankstatements

In [12]:
subscribe!(from(["READ_STMS"]), stm_actor); 

stms = BankStatement[BankStatement(Dates.Date("2020-01-15"), "Duck City Chronicals Invoice A1002", "NL93INGB", 2420.0), BankStatement(Dates.Date("2020-01-15"), "Donalds Hardware Store Bill A1003", "NL39INGB", 1210.0)]


┌ Info: GLActor completed!
└ @ Main /home/rob/projects/TestAppliAR/src/actors.jl:54
┌ Info: GLActor completed!
└ @ Main /home/rob/projects/TestAppliAR/src/actors.jl:54
┌ Info: ARActor Completed!
└ @ Main /home/rob/projects/TestAppliAR/src/actors.jl:41
┌ Info: StmActor completed!
└ @ Main /home/rob/projects/TestAppliAR/src/actors.jl:13


### Display General Leger Accounts Receivable (1300)

You can play with the accountids of the general ledger:
- 1150 Bank
- 1300 Accounts Receivable
- 4000 VAT
- 8000 Sales

In [13]:
using DataFrames

In [20]:
gl = @fetchfrom gl_pid AppliGeneralLedger.read_from_file("./test_ledger.txt")
df = gl |> @filter(_.accountid == 8000) |> DataFrame
df[[:invoice_nbr, :customerid, :debit, :credit, :descr]]

,invoice_nbr,customerid,debit,credit,descr
,String,String,Float64,Float64,String
1,A1001,Scrooge Investment Bank,1210.0,0.0,Learn Smiling
2,A1002,Duck City Chronicals,2420.0,0.0,Learn Smiling
3,A1003,Donalds Hardware Store,1210.0,0.0,Learn Smiling
4,A1002,Duck City Chronicals,0.0,2420.0,Learn Smiling
5,A1003,Donalds Hardware Store,0.0,1210.0,Learn Smiling


### Retrieve Aging Report (Unpaid invoices)

In [18]:
ar = @fetchfrom ar_pid report()
ar |> DataFrame

,id_inv,csm,inv_date,amount,days
,String,String,Dates…,Float64,Dates…
1,A1001,Scrooge Investment Bank,"Date(""2020-12-24"")",1210.0,Day(0)


### Remove the data files from the containers

The SalesActor creates the orders only one time. To repeat the process, the data files have to be remove otherwise you get dublicated records. Keep in mind that also this notebook has to restarted (shutdown and start), unless you don't run cells 4 untill 13 again.

In [16]:
cmd = `ssh rob@172.17.0.2 \ rm test\*`
run(cmd)

Process(`ssh rob@172.17.0.2 ' rm' 'test*'`, ProcessExited(0))

In [17]:
cmd = `ssh rob@172.17.0.3 \ rm test\* invoicenbr.txt`
run(cmd)

Process(`ssh rob@172.17.0.3 ' rm' 'test*' invoicenbr.txt`, ProcessExited(0))